<img src="files/model_1_outline.png">

# change log
### using full data set


sequence generator shape (corrected) (b, n, w, h, c)

In [1]:
import tensorflow as tf
import datetime
import os
import glob
import keras

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [5]:
classes = ['CalotTriangleDissection',
 'CleaningCoagulation',
 'ClippingCutting',
 'GallbladderDissection',
 'GallbladderPackaging',
 'GallbladderRetraction',
 'Preparation']


classes.sort()
print(classes)

['CalotTriangleDissection', 'CleaningCoagulation', 'ClippingCutting', 'GallbladderDissection', 'GallbladderPackaging', 'GallbladderRetraction', 'Preparation']


In [6]:
# some global params
SIZE = (128, 128) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [8]:
# pattern to get videos and classes
glob_pattern='../cholec80/model_data/{classname}/*.avi'


In [9]:
# for data augmentation
# data_aug = keras.preprocessing.image.ImageDataGenerator(
#     zoom_range=.1,
#     horizontal_flip=True,
#     rotation_range=8,
#     width_shift_range=.2,
#     height_shift_range=.2)

In [10]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class CalotTriangleDissection, validation count: 170, train count: 347
class CleaningCoagulation, validation count: 170, train count: 347
class ClippingCutting, validation count: 170, train count: 347
class GallbladderDissection, validation count: 170, train count: 347
class GallbladderPackaging, validation count: 170, train count: 347
class GallbladderRetraction, validation count: 170, train count: 347
class Preparation, validation count: 170, train count: 347
Total data: 7 classes for 2429 files for train


In [11]:
# getting validation data
valid = train.get_validation_generator()

Total data: 7 classes for 1190 files for validation


In [12]:
import keras_video.utils
#keras_video.utils.show_sample(train)

In [13]:
#valid.files

## BUILD CONV NET

In [14]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from keras.applications.mobilenet import MobileNet
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
        
    # adding a max pool
    output = GlobalMaxPool2D()

    return keras.Sequential([model, output])

## Build GRU

In [15]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [16]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 128, 128, 3)
print(INSHAPE)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

#this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

(5, 128, 128, 3)


## epochs, call backs

In [17]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/20210612-080101


In [18]:
EPOCHS=5000
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'model2_3_1_chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/5000
303/303 [==============================] - 188s 585ms/step - loss: 1.8948 - acc: 0.1939 - val_loss: 1.7605 - val_acc: 0.2483

Epoch 00001: saving model to model2_3_1_chkp\weights.01-1.76.hdf5
Epoch 2/5000
303/303 [==============================] - 8s 25ms/step - loss: 1.7100 - acc: 0.2357 - val_loss: 1.5020 - val_acc: 0.2770

Epoch 00002: saving model to model2_3_1_chkp\weights.02-1.50.hdf5
Epoch 3/5000
303/303 [==============================] - 7s 24ms/step - loss: 1.5699 - acc: 0.2773 - val_loss: 2.0670 - val_acc: 0.1850

Epoch 00003: saving model to model2_3_1_chkp\weights.03-2.07.hdf5
Epoch 4/5000
303/303 [==============================] - 7s 24ms/step - loss: 1.5344 - acc: 0.2939 - val_loss: 1.4876 - val_acc: 0.3150

Epoch 00004: saving model to model2_3_1_chkp\weights.04-1.49.hdf5
Epoch 5/5000
303/303 [==============================] - 7s 24ms/step - loss: 1.5541 - acc: 0.3014 - val_loss: 1.3554 - val_acc: 0.3581

Epoch 00005: saving model to model2_3_1_chkp\weights.

303/303 [==============================] - 7s 24ms/step - loss: 0.9245 - acc: 0.5841 - val_loss: 0.8491 - val_acc: 0.6301

Epoch 00041: saving model to model2_3_1_chkp\weights.41-0.85.hdf5
Epoch 42/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.9065 - acc: 0.5761 - val_loss: 1.0787 - val_acc: 0.5270

Epoch 00042: saving model to model2_3_1_chkp\weights.42-1.08.hdf5
Epoch 43/5000
303/303 [==============================] - 7s 24ms/step - loss: 1.0111 - acc: 0.5395 - val_loss: 0.9401 - val_acc: 0.5439

Epoch 00043: saving model to model2_3_1_chkp\weights.43-0.94.hdf5
Epoch 44/5000
303/303 [==============================] - 7s 24ms/step - loss: 1.0487 - acc: 0.5424 - val_loss: 0.8183 - val_acc: 0.6351

Epoch 00044: saving model to model2_3_1_chkp\weights.44-0.82.hdf5
Epoch 45/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.9134 - acc: 0.5860 - val_loss: 1.0108 - val_acc: 0.5642

Epoch 00045: saving model to model2_3_1_chkp\weights.45-1.01.hdf5

303/303 [==============================] - 7s 24ms/step - loss: 0.6119 - acc: 0.7998 - val_loss: 0.5808 - val_acc: 0.8049

Epoch 00081: saving model to model2_3_1_chkp\weights.81-0.58.hdf5
Epoch 82/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.6155 - acc: 0.7826 - val_loss: 0.5745 - val_acc: 0.8167

Epoch 00082: saving model to model2_3_1_chkp\weights.82-0.57.hdf5
Epoch 83/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.6650 - acc: 0.7795 - val_loss: 0.6099 - val_acc: 0.8024

Epoch 00083: saving model to model2_3_1_chkp\weights.83-0.61.hdf5
Epoch 84/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.6129 - acc: 0.8017 - val_loss: 0.5831 - val_acc: 0.8218

Epoch 00084: saving model to model2_3_1_chkp\weights.84-0.58.hdf5
Epoch 85/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.5877 - acc: 0.8120 - val_loss: 0.5985 - val_acc: 0.8117

Epoch 00085: saving model to model2_3_1_chkp\weights.85-0.60.hdf5

Epoch 121/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.4458 - acc: 0.8858 - val_loss: 0.4737 - val_acc: 0.8758

Epoch 00121: saving model to model2_3_1_chkp\weights.121-0.47.hdf5
Epoch 122/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.4658 - acc: 0.8680 - val_loss: 0.4613 - val_acc: 0.8742

Epoch 00122: saving model to model2_3_1_chkp\weights.122-0.46.hdf5
Epoch 123/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.4349 - acc: 0.8788 - val_loss: 0.4703 - val_acc: 0.8657

Epoch 00123: saving model to model2_3_1_chkp\weights.123-0.47.hdf5
Epoch 124/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.4458 - acc: 0.8727 - val_loss: 0.4378 - val_acc: 0.8767

Epoch 00124: saving model to model2_3_1_chkp\weights.124-0.44.hdf5
Epoch 125/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.4463 - acc: 0.8708 - val_loss: 0.4329 - val_acc: 0.8834

Epoch 00125: saving model to model2_3_1_ch

Epoch 161/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.3081 - acc: 0.9125 - val_loss: 0.3437 - val_acc: 0.8986

Epoch 00161: saving model to model2_3_1_chkp\weights.161-0.34.hdf5
Epoch 162/5000
303/303 [==============================] - 7s 23ms/step - loss: 0.2740 - acc: 0.9204 - val_loss: 0.3382 - val_acc: 0.9012

Epoch 00162: saving model to model2_3_1_chkp\weights.162-0.34.hdf5
Epoch 163/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.2951 - acc: 0.9111 - val_loss: 0.3410 - val_acc: 0.8978

Epoch 00163: saving model to model2_3_1_chkp\weights.163-0.34.hdf5
Epoch 164/5000
303/303 [==============================] - 7s 23ms/step - loss: 0.2377 - acc: 0.9323 - val_loss: 0.3672 - val_acc: 0.8902

Epoch 00164: saving model to model2_3_1_chkp\weights.164-0.37.hdf5
Epoch 165/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.2685 - acc: 0.9168 - val_loss: 0.3252 - val_acc: 0.9037

Epoch 00165: saving model to model2_3_1_ch

...

303/303 [==============================] - 7s 24ms/step - loss: 0.1432 - acc: 0.9568 - val_loss: 0.2995 - val_acc: 0.9206

Epoch 04814: saving model to model2_3_1_chkp\weights.4814-0.30.hdf5
Epoch 4815/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1503 - acc: 0.9595 - val_loss: 0.3015 - val_acc: 0.9189

Epoch 04815: saving model to model2_3_1_chkp\weights.4815-0.30.hdf5
Epoch 4816/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1397 - acc: 0.9564 - val_loss: 0.2964 - val_acc: 0.9215

Epoch 04816: saving model to model2_3_1_chkp\weights.4816-0.30.hdf5
Epoch 4817/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1645 - acc: 0.9556 - val_loss: 0.3024 - val_acc: 0.9223

Epoch 04817: saving model to model2_3_1_chkp\weights.4817-0.30.hdf5
Epoch 4818/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1394 - acc: 0.9557 - val_loss: 0.2954 - val_acc: 0.9206

Epoch 04818: saving model to model2_3_1_chkp\weig


Epoch 04853: saving model to model2_3_1_chkp\weights.4853-0.29.hdf5
Epoch 4854/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1812 - acc: 0.9535 - val_loss: 0.2920 - val_acc: 0.9240

Epoch 04854: saving model to model2_3_1_chkp\weights.4854-0.29.hdf5
Epoch 4855/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1460 - acc: 0.9549 - val_loss: 0.3072 - val_acc: 0.9189

Epoch 04855: saving model to model2_3_1_chkp\weights.4855-0.31.hdf5
Epoch 4856/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1336 - acc: 0.9604 - val_loss: 0.2912 - val_acc: 0.9240

Epoch 04856: saving model to model2_3_1_chkp\weights.4856-0.29.hdf5
Epoch 4857/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1311 - acc: 0.9572 - val_loss: 0.3014 - val_acc: 0.9223

Epoch 04857: saving model to model2_3_1_chkp\weights.4857-0.30.hdf5
Epoch 4858/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1296 - acc: 0.9632 - 

303/303 [==============================] - 7s 24ms/step - loss: 0.1663 - acc: 0.9579 - val_loss: 0.3009 - val_acc: 0.9198

Epoch 04893: saving model to model2_3_1_chkp\weights.4893-0.30.hdf5
Epoch 4894/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1388 - acc: 0.9626 - val_loss: 0.3045 - val_acc: 0.9189

Epoch 04894: saving model to model2_3_1_chkp\weights.4894-0.30.hdf5
Epoch 4895/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1671 - acc: 0.9528 - val_loss: 0.3025 - val_acc: 0.9223

Epoch 04895: saving model to model2_3_1_chkp\weights.4895-0.30.hdf5
Epoch 4896/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1704 - acc: 0.9513 - val_loss: 0.2912 - val_acc: 0.9231

Epoch 04896: saving model to model2_3_1_chkp\weights.4896-0.29.hdf5
Epoch 4897/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1510 - acc: 0.9551 - val_loss: 0.2870 - val_acc: 0.9231

Epoch 04897: saving model to model2_3_1_chkp\weig


Epoch 04932: saving model to model2_3_1_chkp\weights.4932-0.30.hdf5
Epoch 4933/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1529 - acc: 0.9519 - val_loss: 0.2909 - val_acc: 0.9223

Epoch 04933: saving model to model2_3_1_chkp\weights.4933-0.29.hdf5
Epoch 4934/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1503 - acc: 0.9511 - val_loss: 0.2964 - val_acc: 0.9215

Epoch 04934: saving model to model2_3_1_chkp\weights.4934-0.30.hdf5
Epoch 4935/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1472 - acc: 0.9458 - val_loss: 0.2947 - val_acc: 0.9198

Epoch 04935: saving model to model2_3_1_chkp\weights.4935-0.29.hdf5
Epoch 4936/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1437 - acc: 0.9572 - val_loss: 0.2962 - val_acc: 0.9215

Epoch 04936: saving model to model2_3_1_chkp\weights.4936-0.30.hdf5
Epoch 4937/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1663 - acc: 0.9524 - 

303/303 [==============================] - 7s 24ms/step - loss: 0.1285 - acc: 0.9577 - val_loss: 0.2943 - val_acc: 0.9215

Epoch 04972: saving model to model2_3_1_chkp\weights.4972-0.29.hdf5
Epoch 4973/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1268 - acc: 0.9616 - val_loss: 0.2870 - val_acc: 0.9257

Epoch 04973: saving model to model2_3_1_chkp\weights.4973-0.29.hdf5
Epoch 4974/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1665 - acc: 0.9575 - val_loss: 0.2884 - val_acc: 0.9223

Epoch 04974: saving model to model2_3_1_chkp\weights.4974-0.29.hdf5
Epoch 4975/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1655 - acc: 0.9561 - val_loss: 0.2991 - val_acc: 0.9215

Epoch 04975: saving model to model2_3_1_chkp\weights.4975-0.30.hdf5
Epoch 4976/5000
303/303 [==============================] - 7s 24ms/step - loss: 0.1281 - acc: 0.9591 - val_loss: 0.3066 - val_acc: 0.9206

Epoch 04976: saving model to model2_3_1_chkp\weig